In [1]:
import openai
from openai.embeddings_utils import cosine_similarity, get_embedding
import os
import numpy as np

In [2]:
import ray.data
import ray
import ray.cloudpickle as pickle

ray.init(ignore_reinit_error=True)

find: ‘.git’: No such file or directory
2023-03-03 11:26:22,865	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.53.95:6379...
2023-03-03 11:26:22,880	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale-staging.com/api/v2/sessions/ses_8y3qzfb6xwlr5ltdjk5vcjh7wm/services?redirect_to=dashboard 
2023-03-03 11:26:23,726	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_1f2ed9b8d62287241f84a0a8f37e0cb5.zip' (1.81MiB) to Ray cluster...
2023-03-03 11:26:23,761	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_1f2ed9b8d62287241f84a0a8f37e0cb5.zip'.


Python version:,3.10.9
Ray version:,3.0.0.dev0
Dashboard:,http://console.anyscale-staging.com/api/v2/sessions/ses_8y3qzfb6xwlr5ltdjk5vcjh7wm/services?redirect_to=dashboard


In [ ]:
print("key is used: ", openai.api_key)

EMB_MODEL = "text-embedding-ada-002"
account_name = "sewerai"
num_sentence_embs = 3
min_num_words_per_sentence = 3
# question = "Why was the user impressed?"
question = "What is their workload? Categorize in batch inference, RLlib, Tune, or Ray serve."

top_k = 10

In [142]:
# DO NOT RUN 
# let's see if this works for out longer text (This is for when we had the entire text)
with open(f"./text_{account_name}.txt", "r") as f:
    text = f.read()
print(text[:100])

Hey Alex. Hey Anthony, how are you? Good, what about you? Doing all right. Good to hear, good to hea


In [4]:
data = ray.data.read_parquet("s3://antoni-test/gong-calls/8269649578171048284.parquet")
df = data.to_pandas()
word_segments = pickle.loads(df["word_segments"].iloc[0])
segments = pickle.loads(df["segments"].iloc[0])

(_get_read_tasks pid=587, ip=10.0.15.173) /tmp/ray/session_2023-03-03_10-16-17_730642_140/runtime_resources/py_modules_files/_ray_pkg_faeda1fd19281a14/ray/data/datasource/parquet_datasource.py:233: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=587, ip=10.0.15.173)   pq_ds.pieces, **prefetch_remote_args
(_get_read_tasks pid=587, ip=10.0.15.173) /tmp/ray/session_2023-03-03_10-16-17_730642_140/runtime_resources/py_modules_files/_ray_pkg_faeda1fd19281a14/ray/data/datasource/parquet_datasource.py:311: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=587, ip=10.0.15.173)   num_files = len(self._pq_ds.pieces)
(_get_read_tasks pid=587, ip=10.0.15.173) /tmp/ray/session_2023-03-03_10-16-17_730642_140/runtime_resources/py_module

In [5]:
word_segments[0:20]

[{'text': 'Hey,', 'start': 0.08036529680365298, 'end': 0.1406392694063927},
 {'text': 'Alex.', 'start': 0.20091324200913246, 'end': 0.4821917808219179},
 {'text': 'Hey,', 'start': 0.5022831050228311, 'end': 1.8283105022831052},
 {'text': 'Anthony.', 'start': 1.9287671232876715, 'end': 2.270319634703197},
 {'text': 'How', 'start': 2.5515981735159823, 'end': 2.6721461187214617},
 {'text': 'are', 'start': 2.7123287671232883, 'end': 2.792694063926941},
 {'text': 'you?', 'start': 2.8328767123287677, 'end': 2.9333333333333336},
 {'text': 'Good.', 'start': 3.5762557077625576, 'end': 3.8173515981735164},
 {'text': 'What', 'start': 3.83744292237443, 'end': 3.9579908675799094},
 {'text': 'about', 'start': 3.998173515981736, 'end': 4.199086757990869},
 {'text': 'you?', 'start': 4.219178082191782, 'end': 4.319634703196348},
 {'text': 'Doing', 'start': 5.961538461538462, 'end': 6.228205128205128},
 {'text': 'all', 'start': 6.2897435897435905, 'end': 6.412820512820513},
 {'text': 'right.', 'start': 

In [6]:
words = [w["text"] for w in word_segments]
timestamps = [(w["start"], w["end"]) for w in word_segments]

['Hey,', 'Alex.', 'Hey,', 'Anthony.', 'How', 'are', 'you?', 'Good.', 'What', 'about']


In [7]:
from dataclasses import dataclass

@dataclass
class Word:
    text: str
    start: float
    end: float

words = [Word(**w) for w in word_segments]
print(words[:10])

[Word(text='Hey,', start=0.08036529680365298, end=0.1406392694063927), Word(text='Alex.', start=0.20091324200913246, end=0.4821917808219179), Word(text='Hey,', start=0.5022831050228311, end=1.8283105022831052), Word(text='Anthony.', start=1.9287671232876715, end=2.270319634703197), Word(text='How', start=2.5515981735159823, end=2.6721461187214617), Word(text='are', start=2.7123287671232883, end=2.792694063926941), Word(text='you?', start=2.8328767123287677, end=2.9333333333333336), Word(text='Good.', start=3.5762557077625576, end=3.8173515981735164), Word(text='What', start=3.83744292237443, end=3.9579908675799094), Word(text='about', start=3.998173515981736, end=4.199086757990869)]


In [99]:
sentences = []
sentence = []

for w in words:
    if w.text.endswith((".", "?", "!")):
        sentence.append(w)
        sentences.append(sentence)
        sentence = []
    else:
        sentence.append(w)
print(sentences[0])
print(sentences[10])

# write the preprocessed sentences into a file to check
preprocessed_sentences_text = "\n\n".join(
    [" ".join([w.text for w in s]) for s in sentences]
)

with open(f"./text_{account_name}_preprocessed.txt", "w") as f:
    f.write(preprocessed_sentences_text)

[Word(text='Hey,', start=0.08036529680365298, end=0.1406392694063927), Word(text='Alex.', start=0.20091324200913246, end=0.4821917808219179)]
[Word(text='Yesterday,', start=17.68019801980198, end=18.4220297029703), Word(text='I', start=18.442079207920795, end=18.462128712871287), Word(text='managed', start=19.38440594059406, end=19.725247524752476), Word(text='to', start=19.765346534653467, end=19.865594059405943), Word(text='avoid', start=19.945792079207923, end=20.246534653465346), Word(text='most', start=20.326732673267326, end=20.56732673267327), Word(text='of', start=20.607425742574257, end=20.66757425742574), Word(text='it,', start=20.747772277227725, end=20.8480198019802), Word(text='but', start=21.128712871287128, end=21.389356435643563), Word(text="it's", start=21.730198019801982, end=22.251485148514853), Word(text='like', start=22.311633663366337, end=22.913118811881187), Word(text='just', start=23.374257425742574, end=23.574752475247525), Word(text='finished', start=23.63490

In [103]:
# DO NOT RUN 
# # split the text by dots and also remove the empty spaces at the beginning and ends
# sentences = [s.strip() for s in text.split(".") if s.strip() != ""]
# remove sentences that are too short (<= 5 words)
# sentences = [s for s in sentences if len(s.split(" ")) > 5]
print("before removing short sentences: ", len(sentences))
# short_setences = []
# for s in sentences:
#     if (len(s)) <= min_num_words_per_sentence:
#         print(str(len(s)) + ", " + " ".join([w.text for w in s]))
#         short_setences.append(str(len(s)) + ", " + " ".join([w.text for w in s]))
#     else:
#         long_sentences.append(s)

# with open(f"./text_{account_name}_short_sentences.txt", "w") as f:
#     f.write("\n\n".join(short_setences))
    
sentences = [s for s in sentences if len(s) > min_num_words_per_sentence]
print("after removing short sentences: ", len(sentences))

text_after_removing_short_sentences = "\n\n".join(
    [" ".join([w.text for w in s]) for s in sentences]
)
with open(f"./text_{account_name}_after_removing_short_sentences.txt", "w") as f:
    f.write(text_after_removing_short_sentences)


# sentences = [" ".join(sentences[i:i+num_sentences]) for i in range(0, len(sentences), num_sentences)]
# merge every num_sentences sentences together consecutively into larger list of words
merged_sentences = []
for i in range(0, len(sentences), num_sentence_embs):
    merged_sentences.append([w for s in sentences[i:i+num_sentence_embs] for w in s])

print("after merging sentences: ", len(merged_sentences))
print("example:")
print(" ".join([w.text for w in merged_sentences[0]]))


batched_sentences = [" ".join([w.text for w in sentence]) for sentence in merged_sentences]
txt_after_processing = "\n\n".join(batched_sentences)

with open(f"./text_{account_name}_processed.txt", "w") as f:
    f.write(txt_after_processing)


before removing short sentences:  186
after removing short sentences:  186
after merging sentences:  62
example:
Has it been raining the last couple of days? Yesterday, I managed to avoid most of it, but it's like just finished raining when I left the office, basically. I'm going to be out there next week, next few weeks.


In [104]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

EMBEDDING_PRICE_PER_1K_TOKENS = 0.0004
full_text = " ".join([w.text for sentence in merged_sentences for w in sentence])
price = num_tokens_from_string(full_text, "cl100k_base") * EMBEDDING_PRICE_PER_1K_TOKENS / 1000
print(price)

0.0013332


In [105]:
resp = openai.Embedding.create(input=batched_sentences, model=EMB_MODEL)

In [106]:
data = resp["data"]
embs = np.array([np.array(d["embedding"]) for d in data])
print(embs)
print(len(embs))


[[-0.02249205 -0.00682723  0.01121285 ... -0.02802549 -0.00293425
  -0.00396431]
 [ 0.00769689  0.00162126  0.02084384 ... -0.02787912 -0.01230193
  -0.00448712]
 [-0.03315589 -0.02292612 -0.01956379 ... -0.01399454  0.00924964
   0.01191743]
 ...
 [-0.02857278 -0.01168145  0.02249667 ... -0.0343476  -0.01708592
  -0.02170577]
 [ 0.00261423 -0.00328275  0.01330881 ... -0.02321178 -0.00982089
  -0.03269072]
 [-0.0121908  -0.00353185  0.00542579 ... -0.00687333 -0.00187425
  -0.0138123 ]]
62


In [107]:
merged_sentences[0]

[Word(text='Has', start=14.148863636363638, end=14.610606060606063),
 Word(text='it', start=14.911742424242426, end=14.971969696969698),
 Word(text='been', start=14.992045454545456, end=15.152651515151517),
 Word(text='raining', start=15.413636363636364, end=15.71477272727273),
 Word(text='the', start=15.734848484848486, end=16.417424242424243),
 Word(text='last', start=16.45757575757576, end=16.598106060606064),
 Word(text='couple', start=16.638257575757578, end=16.798863636363638),
 Word(text='of', start=16.818939393939395, end=16.85909090909091),
 Word(text='days?', start=16.87916666666667, end=17.03977272727273),
 Word(text='Yesterday,', start=17.68019801980198, end=18.4220297029703),
 Word(text='I', start=18.442079207920795, end=18.462128712871287),
 Word(text='managed', start=19.38440594059406, end=19.725247524752476),
 Word(text='to', start=19.765346534653467, end=19.865594059405943),
 Word(text='avoid', start=19.945792079207923, end=20.246534653465346),
 Word(text='most', start

In [108]:
import pandas as pd
sentence_data = [
    {
    "text": " ".join([w.text for w in sentence]), 
    "embedding": embedding,
    "start": sentence[0].start,
    "end": sentence[-1].end,
    } 
    for sentence, embedding in zip(merged_sentences, embs)
]
print(sentence_data[:2])

[{'text': "Has it been raining the last couple of days? Yesterday, I managed to avoid most of it, but it's like just finished raining when I left the office, basically. I'm going to be out there next week, next few weeks.", 'embedding': array([-0.02249205, -0.00682723,  0.01121285, ..., -0.02802549,
       -0.00293425, -0.00396431]), 'start': 14.148863636363638, 'end': 30.91969696969697}, {'text': "And then it's calm here in Sacramento. So like having to commute is the, I'm not, I hope it doesn't, it doesn't rain pretty much in the, during those times. Hi, how's it going?", 'embedding': array([ 0.00769689,  0.00162126,  0.02084384, ..., -0.02787912,
       -0.01230193, -0.00448712]), 'start': 31.281278538812785, 'end': 50.35641025641026}]


In [109]:
pd.DataFrame(sentence_data).to_pickle(f"{account_name}.pkl")

In [110]:
def get_embedding(text: str, model: str = "text-embedding-ada-002") -> np.array:
    """Returns the embedding of a text."""
    resp = openai.Embedding.create(input=[text], model=model)
    emb = np.array(resp["data"][0]["embedding"])
    return emb

In [13]:
# DO NOT RUN
def construct_prompt_for_gpt3(query: str, sentences_emb: np.ndarray, sentences: list,  top_k: int = 10) -> str:
    """Constructs a prompt for the OpenAI API."""
    q_emb = get_embedding(query, model=EMB_MODEL)
    print("q_emb, ", q_emb.shape)
    print("sentences_emb, ", sentences_emb.shape)
    sim_scores = cosine_similarity(sentences_emb, q_emb)
    inds_sorted = np.argsort(sim_scores)[::-1][:top_k]
    print("inds_sorted, ", inds_sorted)
    sentences_sorted = [sentences[i] for i in inds_sorted]
    scores_sorted = [sim_scores[i] for i in inds_sorted]

    context = "\n* ".join(sentences_sorted)
    context = "\n* " + context

    # context = "I've been at Slingshot for about six months now. I'm a machine learning engineer. And I typically focus on my background is actually in more along the lines of software development than is machine learning, though I do have a master's in machine learning or in data science. But so I've been focused mostly on the software side of sort of focusing on the deployment side really of how do we take the take the machine learning models and apply them within our within our applications. So awesome. Yeah. And David, over to you. Yeah. So I'm Dave. Can you hear me all? Yeah. Loud and clear. Yeah. So like Alex, I joined Slingshot about seven months ago or something. And yeah, prior to Slingshot, a lot of my work was in ML, but I was focused on the problems from scientific machine learning to image detection and classification. I did a little bit of reinforcement learning a few years ago, but kind of since I've come to Slingshot, that's been my main area of focus and we've been really trying to stand up some capability to answer some questions, questions for sequential decision making."


    header = """Sahil Lavingia is the founder and CEO of Gumroad, and the author of the book The Minimalist Entrepreneur (also known as TME). These are questions and answers by him. Please keep your answers to three sentences maximum, and speak in complete sentences. Stop speaking once your point is made.\n\nContext that may be useful, pulled from The Minimalist Entrepreneur:\n"""

    qa_examples = [
        "\n\n\nQ: How to choose what business to start?\n\nA: First off don't be in a rush. Look around you, see what problems you or other people are facing, and solve one of these problems if you see some overlap with your passions or skills. Or, even if you don't see an overlap, imagine how you would solve that problem anyway. Start super, super small.",

        "\n\n\nQ: Should we start the business on the side first or should we put full effort right from the start?\n\nA:   Always on the side. Things start small and get bigger from there, and I don't know if I would ever “fully” commit to something unless I had some semblance of customer traction. Like with this product I'm working on now!",

        "\n\n\nQ: Should we sell first than build or the other way around?\n\nA: I would recommend building first. Building will teach you a lot, and too many people use “sales” as an excuse to never learn essential skills like building. You can't sell a house you can't build!",

        "\n\n\nQ: Andrew Chen has a book on this so maybe touché, but how should founders think about the cold start problem? Businesses are hard to start, and even harder to sustain but the latter is somewhat defined and structured, whereas the former is the vast unknown. Not sure if it's worthy, but this is something I have personally struggled with\n\nA: Hey, this is about my book, not his! I would solve the problem from a single player perspective first. For example, Gumroad is useful to a creator looking to sell something even if no one is currently using the platform. Usage helps, but it's not necessary.",

        "\n\n\nQ: What is one business that you think is ripe for a minimalist Entrepreneur innovation that isn't currently being pursued by your community?\n\nA: I would move to a place outside of a big city and watch how broken, slow, and non-automated most things are. And of course the big categories like housing, transportation, toys, healthcare, supply chain, food, and more, are constantly being upturned. Go to an industry conference and it's all they talk about! Any industry…",

        "\n\n\nQ: How can you tell if your pricing is right? If you are leaving money on the table\n\nA: I would work backwards from the kind of success you want, how many customers you think you can reasonably get to within a few years, and then reverse engineer how much it should be priced to make that work.",


        "\n\n\nQ: Why is the name of your book 'the minimalist entrepreneur' \n\nA: I think more people should start businesses, and was hoping that making it feel more “minimal” would make it feel more achievable and lead more people to starting-the hardest step.",

        "\n\n\nQ: How long it takes to write TME\n\nA: About 500 hours over the course of a year or two, including book proposal and outline.",

        "\n\n\nQ: What is the best way to distribute surveys to test my product idea\n\nA: I use Google Forms and my email list / Twitter account. Works great and is 100% free.",

        "\n\n\nQ: How do you know, when to quit\n\nA: When I'm bored, no longer learning, not earning enough, getting physically unhealthy, etc… loads of reasons. I think the default should be to “quit” and work on something new. Few things are worth holding your attention for a long period of time.",
    ]

    prompt = "\n\n\nQ: " + query + "\n\nA: "

    full_prompt = header + context + "".join(qa_examples)  + prompt

    return full_prompt



In [33]:
prompt = construct_prompt_for_gpt3(
    query="What is the use-case?",
    sentences_emb=embs,
    sentences=sentences,
    top_k=10,
)

with open(f"prompt_{account_name}.txt", "w") as f:
    f.write(prompt)

q_emb,  (1536,)
sentences_emb,  (38, 1536)
inds_sorted,  [ 2  3  4 16 17 34 21 18 31  5]


In [30]:
# DO NOT RUN
def run_completion(prompt: str, model: str = "text-davinci-003", max_tokens: int = 100) -> str:
    """Runs a completion on a prompt."""
    resp = openai.Completion.create(
        prompt=prompt,
        engine=model,
        temperature=0.0,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["\n\n"],
    )

    n_tokens = resp["usage"]["total_tokens"]
    price = n_tokens * 0.006 / 1000
    print(f"Number of tokens used for completion: {n_tokens} (${price})")
    return resp["choices"][0]["text"]

In [31]:
# pre-calculate the price of the promopt
num_tokens = num_tokens_from_string(prompt, "gpt2")
num_tokens * 0.006 / 1000

0.009630000000000001

In [ ]:
ans = run_completion(prompt)
print(ans)

In [164]:
def get_most_relevant_segments(query: str, sentences_emb: np.ndarray, sentences: list,  top_k: int = 10) -> list:
    q_emb = get_embedding(query, model=EMB_MODEL)
    sim_scores = cosine_similarity(sentences_emb, q_emb)
    inds_sorted = np.argsort(sim_scores)[::-1][:top_k]
    sentences_sorted = [sentences[i] for i in inds_sorted]
    print("scores:")
    print(sim_scores[inds_sorted])

    return sentences_sorted

def construct_prompt_for_gpt35(query: str, sentences_emb: np.ndarray, sentences: list,  top_k: int = 10, messages=None) -> str:
    """Constructs a prompt for the OpenAI API."""

    if messages is not None:
        if messages[-1]["role"] != "assistant":
            raise ValueError("Last message must be from the assistant")
        
    sentences_sorted = get_most_relevant_segments(
        query, sentences_emb, sentences, top_k=top_k
    )
    
    # number the sentences sorted from 1 ... n
    sentences_sorted_text = [
        f"[{i+1}] ({s['start']:.2f}: {s['end']:.2f}) {s['text']}" for i, s in enumerate(sentences_sorted)
    ]

    system_context = f"""
        You are provided with random parts of a transcript, along with the timestamp in format of (start: end). Your task is to answer questions based on the information in the provided context. Make sure not to use any outside information. When answering, cite the source of the information that you used, even if you are infering it based on multiple sources. For example, if the context is:

        1. John went outside, but Betty stayed at home.
        2. It is raining outside.
        3. On a rainy day, you need an umbrella.

        And the user asks, "Who needed an umbrella?" Your answer should be, "John. Beacuse, it was raining outside [2], and John went outside [1]. so he probably needed an umbrella [3]." 
        The end user will not see the numbered list of sentences, so they won't understand your references. However as the application developer, I will be able to provide them references, if you adhere to the bracket format.

        Here is what we found to be the most relevent based on the query:
        
        {sentences_sorted_text}
    """

    if messages is None:
        messages = [
            {"role": "system", "content": system_context},
            {"role": "user", "content": query}
        ]
    else:
        system_context = f"""
            Here is what we found to be the most relevent based on the new query:
            {sentences_sorted_text}
        """
        messages.append({"role": "system", "content": system_context})
        messages.append({"role": "user", "content": query})
    return messages, sentences_sorted

In [165]:
import re

def run_completion_with_gpt35(messages, sentences_sorted):

    resp = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    assistance_ans = resp["choices"][0]["message"]["content"]
    n_tokens = resp["usage"]["total_tokens"]
    price = n_tokens * 0.002 / 1000
    print(f"Number of tokens used for completion: {n_tokens} (${price})")

    matches = re.findall(r"\[(\d+)\]", assistance_ans)
    
    citations = []
    if matches:
        # minus 1 because the sentences are numbered from 1
        citations = [int(match)-1 for match in matches]

    citations_sentences = [sentences_sorted[i] for i in citations]
    citations_text = [
        f"[{citations[i] + 1}] ({row['start']:.2f}: {row['end']:.2f}) {row['text']}" 
        for i, row in enumerate(citations_sentences)
    ]
    all_sources_text = [f"[{i+1}] ({row['start']:.2f}: {row['end']:.2f}) {row['text']}" for i, row in enumerate(sentences_sorted)]

    if citations_text:
        answer = assistance_ans + "\ncited sources:\n" + "\n".join(citations_text) + "\nAll sources:\n" + "\n".join(all_sources_text)
    else:
        answer = assistance_ans + "\nMost Relevant parts\n" + "\n".join(all_sources_text)
    
    return answer, assistance_ans


In [167]:
print(question)

What is their workload? Categorize in batch inference, RLlib, Tune, or Ray serve.


In [169]:
messages = None
messages, sentences_sorted = construct_prompt_for_gpt35(
    query=question,
    sentences_emb=embs,
    sentences=sentence_data,
    top_k=top_k,
    messages=messages
)
print(messages)
print(sentences_sorted)


scores:
[0.10303652 0.1018212  0.10061335 0.09968505 0.09868071 0.0982319
 0.09806076 0.09746327 0.09696095 0.0966804 ]
[{'role': 'system', 'content': '\n        You are provided with random parts of a transcript, along with the timestamp in format of (start: end). Your task is to answer questions based on the information in the provided context. Make sure not to use any outside information. When answering, cite the source of the information that you used, even if you are infering it based on multiple sources. For example, if the context is:\n\n        1. John went outside, but Betty stayed at home.\n        2. It is raining outside.\n        3. On a rainy day, you need an umbrella.\n\n        And the user asks, "Who needed an umbrella?" Your answer should be, "John. Beacuse, it was raining outside [2], and John went outside [1]. so he probably needed an umbrella [3]." \n        The end user will not see the numbered list of sentences, so they won\'t understand your references. However

In [170]:
answer, assist_answer = run_completion_with_gpt35(messages, sentences_sorted)
print(answer)
messages.append({"role": "assistant", "content": assist_answer})

Number of tokens used for completion: 1066 ($0.002132)
Based on the provided context, it seems like their workload primarily involves batch inference. RLlib, Tune, and Ray serve are not mentioned.
Most Relevant parts
[1] (1325.49: 1345.71) There's any other questions but aside from that, Brent Anthony, a mug, anything else. Just one quick question though like how frequently would these inference jobs be run and like on what data size. But typically, a few hundred a day.
[2] (643.20: 665.25) But that's another common way for batch inference is just every 24 hours every once a week, however often you want to want to do it. That's just another way to have that capability. This is again, very, very neat.
[3] (159.97: 201.82) So I use that example video you have sent as the data. So I have created a bucket with a thousand of those videos for this demo, running 16 of those. So one thing to highlight here is that we are using the new Ray data set streaming executor, which will be available in

In [177]:
messages

[{'role': 'system',
  'content': '\n        You are provided with random parts of a transcript, along with the timestamp in format of (start: end). Your task is to answer questions based on the information in the provided context. Make sure not to use any outside information. When answering, cite the source of the information that you used, even if you are infering it based on multiple sources. For example, if the context is:\n\n        1. John went outside, but Betty stayed at home.\n        2. It is raining outside.\n        3. On a rainy day, you need an umbrella.\n\n        And the user asks, "Who needed an umbrella?" Your answer should be, "John. Beacuse, it was raining outside [2], and John went outside [1]. so he probably needed an umbrella [3]." \n        The end user will not see the numbered list of sentences, so they won\'t understand your references. However as the application developer, I will be able to provide them references, if you adhere to the bracket format.\n\n    

In [184]:
messages[:1]

[{'role': 'system',
  'content': '\n        You are provided with random parts of a transcript, along with the timestamp in format of (start: end). Your task is to answer questions based on the information in the provided context. Make sure not to use any outside information. When answering, cite the source of the information that you used, even if you are infering it based on multiple sources. For example, if the context is:\n\n        1. John went outside, but Betty stayed at home.\n        2. It is raining outside.\n        3. On a rainy day, you need an umbrella.\n\n        And the user asks, "Who needed an umbrella?" Your answer should be, "John. Beacuse, it was raining outside [2], and John went outside [1]. so he probably needed an umbrella [3]." \n        The end user will not see the numbered list of sentences, so they won\'t understand your references. However as the application developer, I will be able to provide them references, if you adhere to the bracket format.\n\n    

In [193]:
question2 = "You mentioned RLlib? Where do they mention RLlib?"
messages, sentences_sorted = construct_prompt_for_gpt35(
    query=question2,
    sentences_emb=embs,
    sentences=sentence_data,
    top_k=top_k,
    messages=messages
)
print(messages)
print(sentences_sorted)


scores:
[0.09523204 0.09482079 0.09444534 0.09425361 0.09396167 0.09392101
 0.09389108 0.09341508 0.09331822 0.09324886]
[{'role': 'system', 'content': '\n        You are provided with random parts of a transcript, along with the timestamp in format of (start: end). Your task is to answer questions based on the information in the provided context. Make sure not to use any outside information. When answering, cite the source of the information that you used, even if you are infering it based on multiple sources. For example, if the context is:\n\n        1. John went outside, but Betty stayed at home.\n        2. It is raining outside.\n        3. On a rainy day, you need an umbrella.\n\n        And the user asks, "Who needed an umbrella?" Your answer should be, "John. Beacuse, it was raining outside [2], and John went outside [1]. so he probably needed an umbrella [3]." \n        The end user will not see the numbered list of sentences, so they won\'t understand your references. Howeve

In [194]:
answer, assist_answer = run_completion_with_gpt35(messages, sentences_sorted)
print(answer)
messages.append({"role": "assistant", "content": assist_answer})

Number of tokens used for completion: 1753 ($0.0035060000000000004)
I apologize for the mistake. After reviewing the context again, I did not find any mention of RLlib.
Most Relevant parts
[1] (845.20: 855.57) And so I wasn't sure how to navigate that situation. I mean, is it. I mean these objects could potentially be you know, 100 megabytes, it's not a crisis if there are two references.
[2] (963.24: 980.83) During, I don't think it's that big a deal I was just curious in terms of better way to handle it. Yeah, so I think like ratio automatically resolve the reference that's being passed into any task. In this case, I guess it's being returned right and you're like yeah.
[3] (773.22: 803.79) What am I saying is I end up with a reference that points to a reference, and this happens frequently with this sort of thing. And so, maybe this wasn't the right example. No, I don't have one here.
[4] (119.12: 145.76) Okay, so we took the liberty of setting up a workspace in your account using t